In [6]:
import numpy as np
import matplotlib.pyplot as plt
from enkf import EnKF

# EKI Testing

## Alex Craig

When applying the EnKF class to an inverse problem there are a few differences:

1. The (augmented) state vector $z$ is defined as:

    $$
    z = ( u, p ) = ( u, G(u) )
    $$

    Where $u$ is the state and $G$ is the forward response operator mapping the unknown $u$ to the observation space. Note that $z$ is of dimension $n + m$ where $n$ is the dimension of the state and $m$ is the dimension of the forward model.

2. The observation operator $H$ is defined as:

    $$
    H = ( 0, I )
    $$

    Where $0$ is a matrix of zeros of dimension $m \times n$ and $I$ is the identity matrix of dimension $m \times m$. This operator is used to extract the forward model from the state vector:

    $$
    H z^k = G(u^k)
    $$

3. The prediction step in KF uses the forward operator F:

    $$
    F(z^{k}) = (u^{k}, G(u^{k})) = (u^{k + 1}, p^{k + 1}) = z^{k + 1}
    $$

4. The observation vector $y$ is defined as:

    $$
    y^{k + 1} = G(u^{k + 1}) + \eta = p^{k + 1} + \eta = H z^{k + 1} + \eta = H z^k + \eta
    $$

    Where $\eta \sim \mathcal{N}(0, R)$ is the observation noise.

    **Note**: Because $y^{k + 1} = H z^k + \eta$ the observation is the same for every iteration.

In [7]:
# Define random seed
seed = 0
np.random.seed(seed)

# Define parameters
k = 100  # Number of time steps
sigma_state = 0.05  # State noise level
sigma_obs = 0.1  # Observation noise level

# State and observation dimensions
n = 2
m = 2
z_dim = n + m  # Total dimension of the augmented state vector z

# Define memory
z = np.zeros((z_dim, k))  # Augmented state vector
y = np.zeros((m, k))  # Observations

# Define the forward response operator G (e.g., a linear model for simplicity)
G = lambda u: 2.0 * u

# Define the augmented state propagation function
def F(z):
    u = z[:n] # Get the first n elements of z
    p = G(u) # Apply the forward operator
    
    z_next = np.concatenate([u, p])
    return z_next

# Observation operator H (extracts the model p from z)
H = np.concatenate([np.zeros((m, n)), np.eye(m)], axis=1)

# Initial state and ensemble members
u0 = np.array([1, 0])
p0 = G(u0)
z0 = np.concatenate([u0, p0])

# Set initial state and observation
z[:, 0] = z0
y[:, 0] = H @ z0

In [8]:
# Set up the true state and observations
# For each time step
for i in range (k - 1):
    # Update state
    z[:,i+1] = F(z[:,i]) + sigma_state * np.random.normal(z_dim)
    
    # Update observation
    y[:,i+1] = H @ z[:,i+1] + sigma_obs * np.random.normal(z_dim)

In [9]:

# Set up inputs for EnFK
# Qsqrt = np.eye(z_dim) * sigma_state
Qsqrt = None
Rsqrt = np.eye(m) * sigma_obs
n_members = 10

# Initialize the EnKF
EnKF = EnKF(F, H, Qsqrt, Rsqrt, seed, n_members=n_members, initialization=None)

print(EnKF.n)

2


In [10]:
# Define memory for the predicted state
z_pred = np.zeros((z_dim, k))

# For each time step
for i in range (k):
    # Get the current observation
    obs = y[:,i]
    
    # Advance the EnKF
    EnKF.advance_ensemble(obs)
     
    # Get the predicted state
    z_pred[:,i] = EnKF.mean

ValueError: could not broadcast input array from shape (4,) into shape (2,)

In [ ]:
# Plotting results
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('True State')
plt.plot(true_state[0, :], true_state[1, :], 'bo-', label='True State u')
plt.plot(true_state[2, :], true_state[3, :], 'go-', label='True State p')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.title('Predicted State')
plt.plot(predicted_states[0, :], predicted_states[1, :], 'ro-', label='Predicted State u')
plt.plot(predicted_states[2, :], predicted_states[3, :], 'mo-', label='Predicted State p')
plt.legend()
plt.grid(True)

plt.show()